In [65]:
from __future__ import absolute_import, division, print_function, unicode_literals

from tensorflow import keras
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

import os
import numpy as np
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, Dropout, MaxPooling1D
 


##### Data preprocessing

In [68]:
def get_class_data(class_name):
    global class_counts
    
    path = os.path.join(train_path, class_name)
    class_units = np.fromfile(path, dtype=int, count=-1, sep=' ', offset=0)
    units = np.split(class_units, class_units.size // 1080, 0) # 1080 = 3 sec * 360Hz
    class_counts[class_name] = class_units.size // 1080
    
    return units

In [71]:
base_path = 'C:\\Users\\User\\Desktop\\1studing\\Курсач 3 курс\\course_work'
train_path = os.path.join(base_path, 'data_files')

classes = ['N', 'LBBB', 'RBBB', 'B', 'T', 'VT', 'SVTA', 'AFIB', 'AFL', 'NOD', 'IVR', 'V']
class_mapping = {'N': 0, 'LBBB': 1, 'RBBB': 2, 'B': 3, 'T': 4, 'VT': 5, 'SVTA': 6, 'AFIB': 7, 'AFL': 8, 'NOD': 9, 'IVR': 10, 'V': 11}
class_counts = {'N': 0, 'LBBB': 0, 'RBBB': 0, 'B': 0, 'T': 0, 'VT': 0, 'SVTA': 0, 'AFIB': 0, 'AFL': 0, 'NOD': 0, 'IVR': 0, 'V': 0}
X = [] # data - each unit 3 sec - 1080 values
Y = [] #labels - from class_mapping

all_data = [] # 12 lists of lists for each class
for class_i in classes:
    class_units = get_class_data(class_i)
    all_data.append(class_units)
    Y  += [class_mapping[class_i]] * class_counts[class_i]
    X += class_units

#print(len(all_data)) #12
#print(len(X)) #8079
#print(len(Y)) #8079


12


##### Build the model

In [62]:
inputs = keras.Input(shape=(1080,)) # change dtype ?

#print(inputs.shape) 
#print(inputs.dtype) tf.float32

#model = Sequential()
#model.add(Conv1D(32, 3, 
#          activation='relu', 
#          input_shape=(8079, 1080)))



model = Sequential()
model.add(Reshape((TIME_PERIODS, num_sensors), input_shape=(input_shape,)))
model.add(Conv1D(100, 10, activation='relu', input_shape=(TIME_PERIODS, num_sensors)))
model.add(Conv1D(100, 10, activation='relu'))
model.add(MaxPooling1D(3))
model.add(Conv1D(160, 10, activation='relu'))
model.add(Conv1D(160, 10, activation='relu'))
model.add(GlobalAveragePooling1D())
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
print(model.summary())

#model = keras.Model(inputs=inputs, outputs=outputs)

output_weight_path = os.path.join(base_path, 'model/cnn_model.hdf5') # or \\ ?
#model.load_weights(output_weight_path, by_name=True) # later - loading pretrained weights ? for testing
model.save_weights(output_weight_path)

##### Training